In [1]:
%load_ext autoreload
%autoreload 2

import tello
import detection
import depth
import time
from tqdm import tqdm
from builtins import classmethod
import multiprocessing
import cv2

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def display_frames(frames):
    fig, ax = plt.subplots(len(frames)//5, 5, figsize=(30,90))
    for rowIdx in range(len(ax)):
        for colIdx in range(len(ax[rowIdx])):
            ax[rowIdx][colIdx].set_axis_off()
            ax[rowIdx][colIdx].imshow(frames[rowIdx*len(ax[rowIdx])+colIdx])
    return fig

class FlightRecording:
    def __init__(self, frames=[]):
        self.fps = 60
        self.fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        self.frames = frames
        
    @classmethod
    def from_file(cls, filename):
        frames = []
        cap = cv2.VideoCapture(filename)
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret:
                frames.append(frame)
            else:
                break
        return cls(frames)
    
    def add_frame(self, frame):
        self.frames.append(frame)
        
    def write(self, filename):
        h, w, c = self.frames[0].shape
        print(f'writing {len(self.frames)} frames to {filename}')
        writer = cv2.VideoWriter(filename, self.fourcc, self.fps, (w, h))
        for frame in self.frames:
            writer.write(frame)
        print('writing done')
        writer.release()
        print('writer released')


def record_drone_video(max_num_frames=100):
    recording = FlightRecording()
    num_frames = 0
    for frame in drone.get_video_frames():
        recording.add_frame(frame)
        print(num_frames)
        num_frames += 1
        if num_frames > max_num_frames:
            break
    recording.write("test.mp4")
    
drone = tello.Tello()
drone.drone.log.log_level = 0
#drone.takeoff()
#drone.drone.clockwise(25)
#drone.land()

Tello: 00:57:08.392:  Info: start video thread
Tello: 00:57:08.392:  Info: send connection request (cmd="conn_req:9617")
Tello: 00:57:08.392:  Info: state transit State::disconnected -> State::connecting
Tello: 00:57:08.397:  Info: video receive buffer size = 425984
Tello: 00:57:08.407:  Info: connected. (port=9617)
Tello: 00:57:08.407:  Info: send_time (cmd=0x46 seq=0x01e4)
Tello: 00:57:08.407:  Info: state transit State::connecting -> State::connected


In [3]:
def record_drone_video2():
    num_frames = 0
    for frame in drone.get_video_frames():
        num_frames += 1
        print(f'{num_frames} frames')
        recording.add_frame(frame)
        if done_flying:
            return

In [4]:
#multiprocessing.Process(target=record_drone_video).start()
time.sleep(10)
starting_flight_info = drone.flight_info
print("taking off")
drone.takeoff()
time.sleep(10)
print("rotating")
drone.rotate('clockwise')
time.sleep(2)
print("rotating")
drone.rotate('clockwise')
time.sleep(2)
print("landing")
drone.land()
#done_flying = True
ending_flight_info = drone.flight_info
drone.drone.quit()

taking off
rotating
rotating
landing


# Recording and CV tests

In [3]:
for frame in drone.get_video_frames():
    detections = detection.detect_objects(frame)
    display(detections)
    print(detections[0].shape)
    depths = depth.detect_depth(frame)
    display(depths)
    print(depths[0].shape)
    break # exit after only one frame
drone.drone.quit()
print("done")

Tello: 22:04:11.243:  Info: get video stream
Tello: 22:04:11.244:  Info: start video (cmd=0x25 seq=0x01e4)
Tello: 22:04:11.248:  Info: recv: ack: cmd=0x34 seq=0x0000 cc 60 00 27 90 34 00 00 00 00 72 a5
Tello: 22:04:11.248:  Info: recv: ack: cmd=0x20 seq=0x0000 cc 60 00 27 b0 20 00 00 00 00 42 b9
Tello: 22:04:11.249:  Info: recv: ack: cmd=0x34 seq=0x0000 cc 60 00 27 90 34 00 00 00 00 72 a5
Tello: 22:04:11.249:  Info: recv: ack: cmd=0x20 seq=0x0000 cc 60 00 27 b0 20 00 00 00 00 42 b9
Tello: 22:04:14.024:  Info: video data 926914 bytes 450.4KB/sec loss=496


non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existing PPS 0 referenced
decode_slice_header error
no frame!
non-existing PPS 0 referenced
non-existi

Tello: 22:04:16.024:  Info: video data 753882 bytes 368.1KB/sec


error while decoding MB 22 40, bytestream -6


Tello: 22:04:18.024:  Info: video data 749724 bytes 366.0KB/sec
Tello: 22:04:20.025:  Info: video data 751160 bytes 366.7KB/sec
Tello: 22:04:22.025:  Info: video data 752430 bytes 367.4KB/sec
Tello: 22:04:24.025:  Info: video data 752639 bytes 367.5KB/sec


...

detections    xmin        ymin        xmax        ymax  confidence  class name
0   0.0  228.576752  955.720093  700.831421    0.322934     59  bed


(array([[[ 18,  81,  94],
         [ 18,  81,  94],
         [ 18,  81,  94],
         ...,
         [128, 176, 155],
         [128, 176, 155],
         [128, 176, 155]],
 
        [[ 18,  81,  94],
         [ 18,  81,  94],
         [ 18,  81,  94],
         ...,
         [128, 176, 155],
         [128, 176, 155],
         [128, 176, 155]],
 
        [[ 18,  81,  94],
         [ 18,  81,  94],
         [ 18,  81,  94],
         ...,
         [131, 179, 158],
         [128, 176, 155],
         [128, 176, 155]],
 
        ...,
 
        [[  0,  57,  91],
         [  0,  57,  91],
         [  0,  57,  91],
         ...,
         [151, 162, 143],
         [153, 164, 145],
         [153, 164, 145]],
 
        [[  0,  57,  91],
         [  0,  57,  91],
         [  0,  57,  91],
         ...,
         [151, 162, 143],
         [152, 163, 144],
         [152, 163, 144]],
 
        [[  0,  57,  91],
         [  0,  57,  91],
         [  0,  57,  91],
         ...,
         [151, 162, 143],
  

(720, 960, 3)
Tello: 22:04:26.026:  Info: video data 746770 bytes 364.4KB/sec
Tello: 22:04:28.027:  Info: video data 759845 bytes 370.9KB/sec
Tello: 22:04:30.027:  Info: video data 749725 bytes 366.1KB/sec loss=2
Tello: 22:04:32.029:  Info: video data 749558 bytes 365.9KB/sec
Tello: 22:04:34.055:  Info: video data 754870 bytes 363.6KB/sec
Tello: 22:04:36.056:  Info: video data 751112 bytes 366.6KB/sec
Tello: 22:04:36.842:  Info: video recv: 1460 bytes 9900 +181
Tello: 22:04:38.061:  Info: video data 755421 bytes 367.9KB/sec
Tello: 22:04:40.089:  Info: video data 759256 bytes 365.7KB/sec
Tello: 22:04:42.089:  Info: video data 752525 bytes 367.4KB/sec


array([[     9.0334,      9.0354,       9.038, ...,      10.136,      9.9921,      9.8985],
       [     9.0415,       9.042,      9.0427, ...,      10.142,      10.071,      10.027],
       [     9.0527,      9.0518,      9.0508, ...,      10.156,      10.193,       10.22],
       ...,
       [     29.925,      29.927,      29.929, ...,      31.922,       32.04,      32.115],
       [     29.911,      29.918,      29.926, ...,      32.317,      32.319,      32.314],
       [       29.9,      29.909,      29.923, ...,      32.573,      32.499,      32.442]], dtype=float32)

(960,)
Tello: 22:04:42.999:  Info: quit
Tello: 22:04:42.999:  Info: state transit State::connected -> State::quit
Tello: 22:04:42.999:  Info: VideoStream.handle_event(DISCONNECTED)
done
Tello: 22:04:43.022:  Info: exit from the video thread.
Tello: 22:04:43.042:  Info: exit from the recv thread.
